This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [68]:
import pandas as pd # standard naming convention
import numpy as np

# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [69]:
N = None
data = pd.read_csv(r'./data/data_000637.txt', nrows = N)
df = pd.DataFrame(data)

df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [70]:
X = df['BX_COUNTER'].mean() #Let's take the mean of all BX
print("The estimated value of X is " + str(round(X)))

The estimated value of X is 1781


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [71]:
import time
import datetime

row_st = df.iloc[0]
time_ns_st = row_st['TDC_MEAS']*25/30 +  row_st['ORBIT_CNT']*row_st['BX_COUNTER']*25

print(str(datetime.timedelta(seconds=time_ns_st*10**-9)))

2 days, 15:47:17.029911


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [78]:

df['TIMENS'] = df['TDC_MEAS'] * 25 / 30 + df['ORBIT_CNT'] * df['BX_COUNTER'] * 25
df['TIMENS'] = df['TIMENS'].cumsum()
#df['TIME_D:H:M:S'] = datetime.timedelta(seconds= float(df['TIMENS']))
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS
0,1,0,123,3869200167,2374,26,2.296370e+14
1,1,0,124,3869200167,2374,27,4.592741e+14
2,1,0,63,3869200167,2553,28,7.062258e+14
3,1,0,64,3869200167,2558,19,9.536611e+14
4,1,0,64,3869200167,2760,25,1.220636e+15
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,2.258180e+20
1310716,1,1,4,3869211171,763,11,2.258181e+20
1310717,1,0,64,3869211171,764,0,2.258182e+20
1310718,1,0,139,3869211171,769,0,2.258183e+20


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [73]:
grouped_TDC = df.groupby('TDC_CHANNEL').sum()

grouped_TDC = grouped_TDC.sort_values("BX_COUNTER", ascending=False)


grouped_TDC.iloc[[0,1,2], [3]]

,BX_COUNTER
TDC_CHANNEL,
139,192477704
64,117767051
63,115482430


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel